# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

Добавим необходимые методы, библиотеки для дальнейшей работы с датасетом и решении поставленной задачи.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

Откроем файл, запишем их в переменную data, проанализируем:

In [ ]:
data = pd.read_csv('/datasets/users_behavior.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Датасет на 3214 записей, можно индефицировать как 3214 клиентов, пропусков нет, чистили его насколько я помню в прошлом модуле, поэтому в предоборотке датасет не нуждается.

### Вывод

Нам достался датасет, с которым мы когда то работали, поэтому предообработка ему не нужна.

Обратимся к вики, чтобы разобраться, что же это такое

Мультиколлинеарность (multicollinearity) — в эконометрике (регрессионный анализ) — наличие линейной зависимости между объясняющими переменными (факторами) регрессионной модели. При этом различают полную коллинеарность, которая означает наличие функциональной (тождественной) линейной зависимости и частичную или просто мультиколлинеарность — наличие сильной корреляции между факторами.

Собственно говоря, как я понял, это сильная взаимосвязь параметров (в нашем случае столбцов), вероятно, можно избавиться от одного для дальнейшего анализа, поскольку все взаимосвязи в другими параметрами будут индентичны, я могу без угрызений совести выкинуть одну из двух фич (минуты или количество звонков). Но делать так не хочется, мало ли что-то сломается в последствии, хотя уверен, что в дальнейшем с этими параметрами не работаю от слова совсем, только загоняю их в признаки на вход модели и от того, что снесу один ничего не поменяется :))

In [ ]:
data.corr(method='spearman')

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.978684,0.155032,0.253886,0.160549
minutes,0.978684,1.000000,0.153784,0.248818,0.159991
messages,0.155032,0.153784,1.000000,0.141999,0.106537
mb_used,0.253886,0.248818,0.141999,1.000000,0.154989
is_ultra,0.160549,0.159991,0.106537,0.154989,1.000000


Да, что и требовалось доказать, для нахождения закономерности с другим столбцами они взаимозаменяемы и это лишняя трата ресурсов вычислительных.

## Разбейте данные на выборки

Разобьем исходные данные на три выборки: обучающую, валидационную и тестовую

В sklearn для этого предусмотрена функция train_test_split. Она разбивает любой датасет на две выборки в зависимости от компонента test_size. Чтобы разбить на три выборки произведем разбиение дважды, сначала разобьем исходные дата сет на обучающую выборку, куда уйдет 60% данных, и другую часть, куда уйдут 40%. Затем разобьем эту часть пополам и половину запишем под валидационную, а другую под тестовую, таким образом у нас получиться 60% обучающей + 20% валидационной + 20% тестовой.

In [ ]:
data_train, data_40 = train_test_split(data, test_size=0.4, random_state=12345)
data_valid, data_test = train_test_split(data_40, test_size=0.5, random_state=12345)

### Вывод
Получили три выборки с разделение 60-20-20 дважды поделив их на две части.

In [ ]:
print(data_train.shape)
print(data_valid.shape)
print(data_test.shape)

(1928, 5)
(643, 5)
(643, 5)


Выведем размерность наших разбиений, как видим, все вышло как надо!

## Исследуйте модели

Прежде чем исследовать модели, необходимо определиться с целевым признаком, для этого нам понадобится бизнес-требование, которое гласит: "Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра»."

Главная задача - это на основе некоторых входных признаков научиться прогназировать, кому более подходит тариф Ультра, а кому нет (данные по нему храняться в столбце "is_ultra", состоящего из 1 и 0 (кстати, столбец разумнее сделать булевым)). Отсюда можно сделать вывод, что перед нами стоит задача бинарной классификации. А целевой признак и есть данный столбец "is_ultra".

Таким образом, нам необходимо создать модель и обучить ее предсказывать данные данного столбца, причем с наименьшей погрешностью. Чтобы минимизировать погрешности, будем перебирать разные модели машинного обучения, которые проходили в данном спринте, а именно:
- Дерево решений
- Случайный лес
- Логистическая регрессия

Мы не просто попробуем разные модели, а попробуем повзаимодействовать с их гиперпараметрами, для достижения более точного результата с минимальными погрешностями.

Так как наборы признаков будут одинаковы для каждой модели, выделим наши features&target для обучения прямо сейчас:

In [ ]:
#data['is_ultra'].unique()
features_train = data_train.drop(['is_ultra'], axis=1)
target_train = data_train['is_ultra']

features_valid = data_valid.drop(['is_ultra'], axis=1)
target_valid = data_valid['is_ultra']

features_test = data_test.drop(['is_ultra'], axis=1)
target_test = data_test['is_ultra']

### Модель Дерево решений

In [ ]:
accuracy_list = [] # список для значений точности

for depth in range(2, 20): # реализуем цикл для подстановки значений в гиперпараметр глубины дерева
    model_tree = DecisionTreeClassifier(random_state=12345, max_depth=depth) # определяем вид модели и ее гиперпараметры

    model_tree.fit(features_train, target_train) # обучаем модель на обучающем наборе данных

    predictions_tree = model_tree.predict(features_valid) # проверяем модель на валидационном наборе данных

    accuracy = round(accuracy_score(target_valid, predictions_tree), 5) # округлим до 5 знаков для читаемости
    accuracy_list.append(accuracy) # добавляем в список точности

print(accuracy_list)
print()
print('Максимальная доля правильных ответов', max(accuracy_list), 'при max_depth =', np.argmax(accuracy_list)+2)

best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=np.argmax(accuracy_list)+2)# сохраним лучший результат

[0.78227, 0.78538, 0.77916, 0.77916, 0.78383, 0.78227, 0.77916, 0.78227, 0.77449, 0.76205, 0.76205, 0.75583, 0.75894, 0.7465, 0.73406, 0.73561, 0.73095, 0.72784]

Максимальная доля правильных ответов 0.78538 при max_depth = 3


Лучший результат достигается при глубине дерева равной трем. Получили точность около 78,5%.

### Модель Случайный лес

В модели Случайный лес будут использоваться два цикла (как и в задании с тренажером), один под глубину дерева, другой под количество деревьев. Двойной цикл не лучшее с точки зрения оптимизации, но что поделать!  

In [ ]:
# соберем значения accuracy и гиперпараметров в списки, из которых потом создадим датафрейм
accuracy_list = []
max_depth_list = []
n_estim_list = []

for depth in range(2, 20):
    for est in range(10, 51, 10): # установим шаг 10, потому что ждать 2 минуты результата - можно взгрустнуть:C
        model_rf = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est) # определяем вид модели и ее гиперпараметры

        model_rf.fit(features_train, target_train) # обучаем модель на обучающем наборе данных

        predictions_rf = model_rf.predict(features_valid) # проверяем модель на валидационном наборе данных

        accuracy = round(accuracy_score(target_valid, predictions_rf), 5) # округлим до 5 знаков для читаемости

        accuracy_list.append(accuracy)
        max_depth_list.append(depth)
        n_estim_list.append(est)

# собираем все списки в один
result_list = [max_depth_list, n_estim_list, accuracy_list]

# создаем датафрейм
data_random_forest = pd.DataFrame({'max_depth':max_depth_list, 'n_estimators':n_estim_list, 'accuracy':accuracy_list})

# выводим на экран строку с наибольшей точностью
display(data_random_forest[data_random_forest['accuracy']==data_random_forest['accuracy'].max()])

best_forest_model = RandomForestClassifier(random_state=12345, max_depth=8, n_estimators=40) # сохраним по результатам прогона

,max_depth,n_estimators,accuracy
33,8,40,0.80871


Самый лучший результат получили при глубине дерева 15 и количестве деревьев 50. Доля правильных ответов составляет более 80%.

### Модель Логистическая регрессия

In [ ]:
for solve in ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']:
    model = LogisticRegression(random_state=12345, solver=solve)

    model.fit(features_train, target_train)

    predictions = model.predict(features_valid)

    accuracy_lr = round(accuracy_score(target_valid, predictions), 5)
    print(accuracy_lr)

best_reg_model = LogisticRegression(random_state=12345, solver='newton-cg')

0.75583
0.71073
0.70918
0.70607
0.70607


Нашел в документации различные возможные параметры гиперпараметра solver, это вероятно выбранный алгоритм для логистической регрессии, пока что не разобрался наверняка как он работает, но для разнообразия перебрал его, и как мы видим, выбил этим самым результат >75%. Правда красные варнинги очень настораживают ...

### Вывод по главе:
Таким образом мы можем выделить наиболее удачные модели, а именно дерево решений с глубиной 3, случайный лес с глубиной 15 и количеством деревьев равным 50, а так же логистическую регрессию с параметром solver='newton-cg'.

## Проверьте модель на тестовой выборке

In [ ]:
best_tree_model.fit(features_train, target_train)
test_predictions = best_tree_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели дерева решений на тестовой выборке',accuracy)

best_forest_model.fit(features_train, target_train)
test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели случайного леса на тестовой выборке',accuracy)

best_reg_model.fit(features_train, target_train)
test_predictions = best_reg_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
print('Точность модели логистической регресиии на тестовой выборке',accuracy)

Точность модели дерева решений на тестовой выборке 0.7791601866251944
Точность модели случайного леса на тестовой выборке 0.7962674961119751
Точность модели логистической регресиии на тестовой выборке 0.7387247278382582


Снова варнинги:C Немного костыльно выглядит, снова пришлось прописывать обучение, так как я записывал отдельно лучшие модели, но вроде работает, лучшая точность у Случайного леса, худшая у логистической регрессии.

## (бонус) Проверьте модели на адекватность

Ответ на данный раздел я нашел в глубинах интернета, хотя как глубинах, минут 10, в целом понял (хотя узнал, что такоее среднее гармоническое), так что теперь смогу провернуть это для любой задачи классификации.

Оценить адекватность модели можно используя несколько метрик:

- accuracy - точность совпадения прогнозов с правильными ответами.
- precision - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 1(FP).
- recall - отношение правильно предсказанных классов равных 1(TP) к сумме правильно предсказанных классов равных 1(TP) с неправльными предсказаниями, указанные как 0(FN).

Для оценки адеватности используется F-мера - среднее гармоническое между precision и recall. Если хотя бы один из параметров близок к нулю, то и F-мера стремится к 0. Если оба стремятся к 1, то F-мера тоже стремится к 1.

In [ ]:
test_predictions = best_forest_model.predict(features_test)
accuracy = accuracy_score(target_test, test_predictions)
precision = precision_score(target_test, test_predictions)
recall = recall_score(target_test, test_predictions)
print('Accuracy =', accuracy, 'Precision =', precision, 'Recall =', recall)

f_score = f1_score(target_test, test_predictions)
print('F-мера =', f_score)

Accuracy = 0.7962674961119751 Precision = 0.75 Recall = 0.5320197044334976
F-мера = 0.622478386167147


### Вывод

Адекватность модели оценивается по нескольким параметрам. Для модели классификации это accuracy, precision и recall. Качество модели лучше всего отражают precision и recall и эти метрики складываются в F-мера как единую оценку модели.

Модель получилась среднего качества. И для таких задач она адекватна.